In [1]:
import pandas as pd

# from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB
import re
import nltk
from nltk.corpus import stopwords

SEED = 3

In [4]:
df = pd.read_csv('../data/ToLD-BR.csv')
df.fillna(0, inplace=True)
df

,text,homophobia,obscene,insult,racism,misogyny,xenophobia
0,Meu nivel de amizade com isis é ela ter meu in...,0.0,0.0,2.0,0.0,0.0,0.0
1,"rt @user @user o cara adultera dados, que fora...",0.0,0.0,1.0,0.0,0.0,0.0
2,@user @user @user o cara só é simplesmente o m...,0.0,2.0,1.0,0.0,0.0,0.0
3,eu to chorando vei vsf e eu nem staneio izone ...,0.0,1.0,0.0,0.0,0.0,0.0
4,Eleitor do Bolsonaro é tão ignorante q não per...,0.0,1.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
20995,@user faz favor vai carai,0.0,0.0,0.0,0.0,0.0,0.0
20996,só queria conhecer alguém que não conhece o he...,1.0,0.0,0.0,0.0,0.0,0.0
20997,"vcs militam na hora errada em cima de memes, p...",0.0,0.0,0.0,0.0,0.0,0.0
20998,@user porra any eu tava c dor de cabeca e fui ...,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\laura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
def preprocess_text(text):
    if not isinstance(text, str):
        text = str(text)
    # Remover IDs e padrões >>, mantendo expressões
    text = re.sub(r'>>\d+', '', text)
    # Remover menções como @usuário
    text = re.sub(r'@\w+', '', text)
    # Manter emojis, pontos de interrogação e exclamação
    text = re.sub(r'[^\w\s!?]', '', text)
    # Converter para minúsculas
    text = text.lower()
    return text

df['txt_clean'] = df['text'].apply(preprocess_text)
df.drop(columns=['text'], inplace=True)
display(df)

,homophobia,obscene,insult,racism,misogyny,xenophobia,txt_clean
0,0.0,0.0,2.0,0.0,0.0,0.0,meu nivel de amizade com isis é ela ter meu in...
1,0.0,0.0,1.0,0.0,0.0,0.0,rt o cara adultera dados que foram desmascar...
2,0.0,2.0,1.0,0.0,0.0,0.0,o cara só é simplesmente o maior vencedor d...
3,0.0,1.0,0.0,0.0,0.0,0.0,eu to chorando vei vsf e eu nem staneio izone ...
4,0.0,1.0,2.0,0.0,0.0,0.0,eleitor do bolsonaro é tão ignorante q não per...
...,...,...,...,...,...,...,...
20995,0.0,0.0,0.0,0.0,0.0,0.0,faz favor vai carai
20996,1.0,0.0,0.0,0.0,0.0,0.0,só queria conhecer alguém que não conhece o he...
20997,0.0,0.0,0.0,0.0,0.0,0.0,vcs militam na hora errada em cima de memes po...
20998,0.0,0.0,0.0,0.0,0.0,0.0,porra any eu tava c dor de cabeca e fui comer...


### **Exploratory Data Analysis (EDA)**

In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7672 entries, 0 to 7671
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   has_anger  7672 non-null   int32 
 1   txt_clean  7672 non-null   object
dtypes: int32(1), object(1)
memory usage: 90.0+ KB
None


In [6]:
print(df['has_anger'].value_counts())

has_anger
1    3836
0    3836
Name: count, dtype: int64


In [7]:
df.describe()

,has_anger
count,7672.000000
mean,0.500000
std,0.500033
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


In [8]:
df['txt_clean']

0       apóio o  um passo de cada vez não tenha pressa...
1                        eu ainda vou surtar com essa fic
2        kkkkkkkkkkkk amei e fiquei com vontade desse ...
3       debater com luquistajá cansei de bater palma p...
4            hoje eu só tô minha irmã ali kkkkkkkkkkkkkk 
                              ...                        
7667                            neozelandesas culias !!!!
7668    2011prosperidade é boa2019prosperidade é uma m...
7669    renovador seu sorriso derretia satélites e cor...
7670    fotos lindas que não podem ser postadas lament...
7671    só tô esperando uma oportunidade sua  senta se...
Name: txt_clean, Length: 7672, dtype: object

In [9]:
df_anger = df[df['has_anger'] == 1]
print(df_anger)

      has_anger                                          txt_clean
0             1  apóio o  um passo de cada vez não tenha pressa...
3             1  debater com luquistajá cansei de bater palma p...
5             1  ela me chamou pra dormir na casa dela pois est...
8             1  seu psiquiatra parece ter errado muito provave...
15            1  ah anão eu te invejo sei que tem o fator psico...
...         ...                                                ...
7657          1  só estou falando a verdadenão quero me estende...
7661          1  se não posso ter uma menina de 10 e 11 anos co...
7664          1  25 anosquitei minha seguna faculdadesem empreg...
7665          1  não sei vocês mas acho tudo isso muito muito f...
7668          1  2011prosperidade é boa2019prosperidade é uma m...

[3836 rows x 2 columns]


In [10]:
df_anger = df[df['has_anger'] == 0]
print(df_anger['txt_clean'])

1                        eu ainda vou surtar com essa fic
2        kkkkkkkkkkkk amei e fiquei com vontade desse ...
4            hoje eu só tô minha irmã ali kkkkkkkkkkkkkk 
6                         a co delas ve volnem case?\nja 
7                                               obrigado 
                              ...                        
7666    depois q descobri minha paixão por gloss nunca...
7667                            neozelandesas culias !!!!
7669    renovador seu sorriso derretia satélites e cor...
7670    fotos lindas que não podem ser postadas lament...
7671    só tô esperando uma oportunidade sua  senta se...
Name: txt_clean, Length: 3836, dtype: object


In [11]:
print(df.iloc[0][0], f'Hate.speech: {df.iloc[0][1]}', sep='\n')
print('\n')
print(df.iloc[74][0], f'Hate.speech: {df.iloc[74][1]}', sep='\n')

1
Hate.speech: apóio o  um passo de cada vez não tenha pressa mas sempre movendo pra frente e não tenha medo de pedir ajuda a amigos parentes ou psicólogos não tem nada de maise outra faculdade nem sempre é a solução tem cursos técnicos que também contam como superior e que às vezes são mais curtos que faculdades e te pagam melhor


0
Hate.speech:  foi ela mesmo minha mãe olhou na geladeira dela porca ladra de comida


C:\Users\laura\AppData\Local\Temp\ipykernel_940\2683638277.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df.iloc[0][0], f'Hate.speech: {df.iloc[0][1]}', sep='\n')
C:\Users\laura\AppData\Local\Temp\ipykernel_940\2683638277.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df.iloc[74][0], f'Hate.speech: {df.iloc[74][1]}', sep='\n')


### **Model training**

#### **Preprocessing**

In [12]:
X = df['txt_clean']
y = df['has_anger']

In [13]:
print(f'X nulls: {X.isna().sum()}')
print(f'y nulls: {y.isna().sum()}')

X nulls: 0
y nulls: 0


In [14]:
def get_pipe(model, params: dict):
    return Pipeline(steps=[
        ('vectorizer', TfidfVectorizer()),
        ('model', model(**params)),
    ])

def validate_pipe(pipe, X, y):
    accuracy = cross_val_score(pipe, X, y, cv=StratifiedKFold(n_splits=5), scoring='accuracy')
    precision = cross_val_score(pipe, X, y, cv=StratifiedKFold(n_splits=5), scoring='precision')
    recall = cross_val_score(pipe, X, y, cv=StratifiedKFold(n_splits=5), scoring='recall')
    f1 = cross_val_score(pipe, X, y, cv=StratifiedKFold(n_splits=5), scoring='f1')

    result_matrix = pd.DataFrame.from_dict({
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    })

    print(
        f'Accuracy: {result_matrix['accuracy'].mean():.1%}',
        f'Precision: {result_matrix['precision'].mean():.1%}',
        f'Recall: {result_matrix['recall'].mean():.1%}',
        f'F1: {result_matrix['f1'].mean():.1%}',
        sep='\n',
    )

    return result_matrix

In [15]:
multinomialnb_pipe = get_pipe(MultinomialNB, {})
validate_pipe(multinomialnb_pipe, X, y)

Accuracy: 72.2%
Precision: 64.4%
Recall: 99.7%
F1: 78.2%


,accuracy,precision,recall,f1
0,0.727687,0.648305,0.996094,0.785421
1,0.708795,0.632231,0.997392,0.773900
2,0.724250,0.645025,0.997392,0.783410
3,0.737288,0.656627,0.994785,0.791083
4,0.713820,0.636439,0.997392,0.777044


In [16]:
complementnb_pipe = get_pipe(ComplementNB, {})
validate_pipe(complementnb_pipe, X, y)

Accuracy: 72.5%
Precision: 64.6%
Recall: 99.7%
F1: 78.4%


,accuracy,precision,recall,f1
0,0.727687,0.648305,0.996094,0.785421
1,0.721173,0.642317,0.997392,0.781410
2,0.724250,0.645025,0.997392,0.783410
3,0.737288,0.656627,0.994785,0.791083
4,0.713820,0.636439,0.997392,0.777044


In [17]:
bernoullinb_pipe = get_pipe(BernoulliNB, {})
validate_pipe(bernoullinb_pipe, X, y)

Accuracy: 83.7%
Precision: 99.8%
Recall: 67.4%
F1: 80.5%


,accuracy,precision,recall,f1
0,0.846906,0.996276,0.696615,0.819923
1,0.842345,0.998102,0.685789,0.812983
2,0.828553,1.000000,0.657106,0.793076
3,0.829205,0.998028,0.659713,0.794349
4,0.836375,1.000000,0.672751,0.804365
